In [ ]:
import numpy as np
import pandas as pd
import requests

#每日進站人口數,政府開放平台 - https://data.gov.tw/dataset/8792
url1 = 'https://ods.railway.gov.tw/tra-ods-web/ods/download/dataResource/8ae4cabf6973990e0169947ed32454b9'

#車站基本資料集 - https://data.gov.tw/dataset/33425
url2 = 'https://ods.railway.gov.tw/tra-ods-web/ods/download/dataResource/0518b833e8964d53bfea3f7691aea0ee'

response = requests.get(url1)
jsonData = response.json()
jsonData

In [3]:
#建立DataFrame
#trnOpDate(乘車日)、staCode(車站代碼)、gateInComingCnt(進站人數)、gateOutGoingCnt(出站人數)
dataFrame = pd.DataFrame(jsonData)
dataFrame.columns = ['乘車日','車站代碼','進站人數','出站人數']
dataFrame

,乘車日,車站代碼,進站人數,出站人數
0,20220101,0900,8645,8526
1,20220101,0910,996,1243
2,20220101,0920,1369,1711
3,20220101,0930,3663,4535
4,20220101,0940,1532,1845
...,...,...,...,...
64979,20221001,7360,4163,4395
64980,20221001,7361,161,243
64981,20221001,7362,278,405
64982,20221001,7380,495,515


In [ ]:
#下載車站基本資料
response = requests.get(url2)
stations_json = response.json()
stations_json

In [9]:
#stationCode(車站代碼)、stationName(中文站名)、stationEName(英文站名)、name(網站中文站名)、ename(網站英文站名)、stationAddrTw(車站地址)、stationAddrEn(車站英文地址)、stationTel(車站電話)、gps
#車站DataFrame
stataions = pd.DataFrame(stations_json,columns=['stationCode','stationName'])
stataions.columns=["車站代碼","中文站名"]
stataions

,車站代碼,中文站名
0,0900,基隆
1,0910,三坑
2,0920,八堵
3,0930,七堵
4,0940,百福
...,...,...
237,7360,瑞芳
238,7361,海科館
239,7362,八斗子
240,7380,四腳亭


In [13]:
#pd.merge()
#1對多
mergeFrame = pd.merge(dataFrame,stataions,on='車站代碼',how='left')
mergeFrame

,乘車日,車站代碼,進站人數,出站人數,中文站名
0,20220101,0900,8645,8526,基隆
1,20220101,0910,996,1243,三坑
2,20220101,0920,1369,1711,八堵
3,20220101,0930,3663,4535,七堵
4,20220101,0940,1532,1845,百福
...,...,...,...,...,...
64979,20221001,7360,4163,4395,瑞芳
64980,20221001,7361,161,243,海科館
64981,20221001,7362,278,405,八斗子
64982,20221001,7380,495,515,四腳亭


In [15]:
#檢查
mergeFrame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64984 entries, 0 to 64983
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   乘車日     64984 non-null  object
 1   車站代碼    64984 non-null  object
 2   進站人數    64984 non-null  object
 3   出站人數    64984 non-null  object
 4   中文站名    64984 non-null  object
dtypes: object(5)
memory usage: 3.0+ MB


In [19]:
#轉換型別
mergeFrame['進站人數'] = mergeFrame['進站人數'].astype(np.int32)
mergeFrame['出站人數'] = mergeFrame['出站人數'].astype(np.int32)

In [20]:
#重塑
mergeFrame1 = mergeFrame.set_index(["乘車日","中文站名"])
mergeFrame1

車站代碼  進站人數  出站人數
乘車日      中文站名                  
20220101 基隆    0900  8645  8526
         三坑    0910   996  1243
         八堵    0920  1369  1711
         七堵    0930  3663  4535
         百福    0940  1532  1845
...             ...   ...   ...
20221001 瑞芳    7360  4163  4395
         海科館   7361   161   243
         八斗子   7362   278   405
         四腳亭   7380   495   515
         暖暖    7390   355   411

[64984 rows x 3 columns]

In [21]:
#儲存為execel
mergeFrame1.to_excel('excel1.xlsx')

In [27]:
#朔型
mergeFrame2 = mergeFrame1.drop('車站代碼',axis=1).stack().unstack(level='中文站名')
mergeFrame2

中文站名               七堵      三坑    三塊厝     三姓橋    三民     三義    三貂嶺    上員  \
乘車日                                                                      
20220101 進站人數  3663.0   996.0  645.0   755.0   7.0  784.0  147.0  62.0   
         出站人數  4535.0  1243.0  559.0   797.0  12.0  944.0  216.0  84.0   
20220102 進站人數  3691.0  1011.0  495.0   835.0  12.0  975.0  140.0  85.0   
         出站人數  3773.0  1007.0  496.0  1096.0  17.0  725.0  104.0  91.0   
20220103 進站人數  6042.0  1337.0  566.0  1175.0   2.0  590.0   35.0  49.0   
...               ...     ...    ...     ...   ...    ...    ...   ...   
20220929 出站人數  5684.0  1340.0  575.0  1055.0   NaN  524.0  212.0  47.0   
20220930 進站人數  6269.0  1371.0  556.0  1562.0   5.0  612.0  138.0  58.0   
         出站人數  6350.0  1425.0  657.0  1159.0   8.0  783.0  133.0  60.0   
20221001 進站人數  3975.0  1014.0  464.0   780.0   3.0  500.0  501.0  61.0   
         出站人數  3937.0   757.0  425.0   712.0   3.0  711.0  395.0  60.0   

中文站名                中壢     中洲  ...      鳳山     鳳林      鶯歌     鹿野     麟洛  \
乘車日                            ...                                        
20220101 進站人數  24577.0  372.0  ...  4563.0  279.0  7345.0  201.0  182.0   
         出站人數  28729.0  386.0  ...  4734.0  279.0  8169.0  159.0  217.0   
20220102 進站人數  25927.0  348.0  ...  5789.0  482.0  6509.0  345.0  196.0   
         出站人數  27533.0  464.0  ...  6187.0  353.0  6258.0  154.0  170.0   
20220103 進站人數  20276.0  371.0  ...  4712.0  366.0  7845.0  241.0  148.0   
...                ...    ...  ...     ...    ...     ...    ...    ...   
20220929 出站人數  18677.0  331.0  ...  3921.0  284.0  7895.0  103.0  118.0   
20220930 進站人數  24482.0  425.0  ...  5438.0  277.0  8396.0   95.0  141.0   
         出站人數  23319.0  379.0  ...  5521.0  454.0  8484.0  146.0  171.0   
20221001 進站人數  20858.0  328.0  ...  4184.0  224.0  6458.0   95.0  131.0   
         出站人數  20880.0  339.0  ...  3753.0  257.0  6344.0   89.0  162.0   

中文站名              鼓山     龍井    龍泉    龍港     龜山  
乘車日                                             
20220101 進站人數  550.0  425.0  16.0  25.0  110.0  
         出站人數  392.0  461.0  31.0  25.0   97.0  
20220102 進站人數  411.0  464.0  17.0  19.0   87.0  
         出站人數  414.0  384.0  31.0  27.0   48.0  
20220103 進站人數  261.0  343.0  10.0  14.0   67.0  
...              ...    ...   ...   ...    ...  
20220929 出站人數  277.0  268.0   7.0  23.0   84.0  
20220930 進站人數  401.0  311.0  11.0  12.0   66.0  
         出站人數  369.0  400.0   2.0  23.0   71.0  
20221001 進站人數  380.0  319.0  10.0  17.0   70.0  
         出站人數  372.0  252.0   9.0  16.0   78.0  

[548 rows x 239 columns]

In [28]:
mergeFrame2.to_excel('excel2.xlsx')

In [29]:
mergeFrame1

車站代碼  進站人數  出站人數
乘車日      中文站名                  
20220101 基隆    0900  8645  8526
         三坑    0910   996  1243
         八堵    0920  1369  1711
         七堵    0930  3663  4535
         百福    0940  1532  1845
...             ...   ...   ...
20221001 瑞芳    7360  4163  4395
         海科館   7361   161   243
         八斗子   7362   278   405
         四腳亭   7380   495   515
         暖暖    7390   355   411

[64984 rows x 3 columns]

In [37]:
#取出1月乘車日
frame1 = mergeFrame1.loc['20220101':'20220131'].groupby(level='中文站名').sum(True)
frame1=frame1.reset_index()
frame1['2022'] = '1月份'
frame1=frame1.set_index(['2022','中文站名'])
frame1

進站人數    出站人數
2022 中文站名                
1月份  七堵    149018  150782
     三坑     35238   36927
     三塊厝    15892   16474
     三姓橋    25332   23397
     三民       150     327
...           ...     ...
     鼓山      7699    7428
     龍井      9029    8905
     龍泉       344     803
     龍港       473     687
     龜山      1945    2004

[239 rows x 2 columns]

In [46]:
#取出2月乘車日
frame2 = mergeFrame1.loc['20220201':'20220231'].groupby(level='中文站名').sum(True)
frame2=frame2.reset_index()
frame2['2022'] = '2月份'
frame2=frame2.set_index(['2022','中文站名'])
frame2

進站人數    出站人數
2022 中文站名                
2月份  七堵    126258  126767
     三坑     31190   32514
     三塊厝    13299   13364
     三姓橋    23340   21178
     三民       137     339
...           ...     ...
     鼓山      7758    7487
     龍井      8636    8256
     龍泉       298     377
     龍港       429     602
     龜山      1916    1797

[239 rows x 2 columns]

In [47]:
result = pd.concat([frame1,frame2])
result

進站人數    出站人數
2022 中文站名                
1月份  七堵    149018  150782
     三坑     35238   36927
     三塊厝    15892   16474
     三姓橋    25332   23397
     三民       150     327
...           ...     ...
2月份  鼓山      7758    7487
     龍井      8636    8256
     龍泉       298     377
     龍港       429     602
     龜山      1916    1797

[478 rows x 2 columns]

In [51]:
#重朔
result1=result.stack().unstack(level=['2022',2])
result1

2022     1月份             2月份        
        進站人數    出站人數    進站人數    出站人數
中文站名                                
七堵    149018  150782  126258  126767
三坑     35238   36927   31190   32514
三塊厝    15892   16474   13299   13364
三姓橋    25332   23397   23340   21178
三民       150     327     137     339
...      ...     ...     ...     ...
鼓山      7699    7428    7758    7487
龍井      9029    8905    8636    8256
龍泉       344     803     298     377
龍港       473     687     429     602
龜山      1945    2004    1916    1797

[239 rows x 4 columns]

In [50]:
result1.to_excel('excel3.xlsx')